In [1]:
#the simpsons
#american horror story
#how i met your mother
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

#전역 변수
url = 'https://www.springfieldspringfield.co.uk/episode_scripts.php?'
script_list = []   #다른 함수에서도 써야하기 때문에 전역변수로 선언
episode_list = []

#입력 받은 값에서 띄어쓰기가 있는 경우 '-'로 변환
def replace_space(series_name):  
    
    series_name = series_name.replace(" ", "-") #띄어쓰기는 url주소에 쓰이기 위해서 '-'로 변환
    series_name.lower()    #소문자로 바꾸기
    return series_name



#대본 크롤링
def script_crawling(series_name):
    params = {
        'tv-show' : series_name
    }

    resp = requests.get(url, params = params)
    soup = BeautifulSoup(resp.content, 'html.parser')
    
    find_frame = soup.find('div', class_ = 'main-content-left')           #가장 바깥 프레임 설정
    find_seasons = find_frame.find_all('div', class_ = 'season-episodes') #모든 시즌 가져오기
                                                                          #find_seasons[0] = season1

    url_list = []                                     #각 에피소드들의 주소를 담을 리스트
    url_base = 'https://www.springfieldspringfield.co.uk/'  
    for season in find_seasons:                            
        #에피소드 태그 추출               
        a_tags = season.find_all('a')    
                                      
        for a in a_tags:
            url_tag = a.attrs['href']                #에피소드 별 주소 추출            
            url_tag = url_base + url_tag             # 대본으로 가는 url 합치기        
            url_list.append(url_tag)                 #에피소드 대본 url a_tag_list에 넣기            

    print(url_list)

    #https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e01  's01e01' 만 빼오기    
    for i in url_list:    
        episode = i[-5:] 
        episode_list.append(episode)
    print(episode_list)
    
    #에피소드별 url에 접속 후 대본 가져오기

    count = 0
    for script_url in url_list:           #seasons_list 각 인덱스에 있는 url에 접속
        script_resp = requests.get(script_url)
        script_soup = BeautifulSoup(script_resp.content, 'html.parser')
        script_list.append(script_soup.find('div', class_ = 'scrolling-script-container').text)  #대본 페이지 본문 긁어 오기
        
        #제한 설정: 10개 에피소드 긁어오기
        count += 1
        if count == 10:
            break
#script crwaling end 
            
            
#텍스트 파일 하나로 몰아넣기
def save_to_txt(series_name):
    # 처음에 입력 받은 이름으로 파일 저장하기
    file_name = series_name + '_script.txt'
    print(file_name)
    count = 0
    with open(file_name, 'w', -1, 'utf-8') as f:   #'cp949' codec can't encode character '\xe8' in position 11518: 
                                                       #illegal multibyte sequence 에러 때문에 'utf-8' 인코딩 수정
        f.write('%s\n' % series_name)              # 미드 이름
        for scripts in script_list:                #해당 시즌 모든 에피소드 텍스트 파일에 입력
            f.write('\n-----------------------------------%s' % episode_list[count])
            f.write('\n%s' % scripts)
            count += 1            

#미드 이미지 가져오기/et(img_url)
def get_img(series_name):

    driver_path = 'C:\\Users\\15Z970-GA5BK\\Downloads\\chromedriver.exe'
    driver = webdriver.Chrome(driver_path)
    
    driver.get('https://www.google.com')
    driver.implicitly_wait(10)
    search_input = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[1]/div/div[1]/input')
    search_input.send_keys(series_name)

    search_btn = driver.find_element_by_xpath('//*[@id="tsf"]/div[2]/div/div[3]/center/input[1]').click()
    driver.implicitly_wait(10)
    image_btn = driver.find_element_by_link_text('이미지').click()
    
    #구글 이미지 
    #다음 페이지 이미지 불러오기
    img_count = 0   
    img_url_list = []
    while img_count < 40:
        img_next_url = 'https://www.google.com/search?q=' + series_name + \
                         '&biw=1036&bih=674&gbv=1&tbm=isch&ei=jWv6XKqsLo3VmAXk8ILQBg&start=' + str(img_count)
        # url에서 'start=' 값이 20씩 늘어날 때마다 다음 페이지로 넘어감
        resp3 = requests.get(img_next_url)
        soup3 = BeautifulSoup(resp3.content, 'html.parser')
        find_img = soup3.find('table', class_ = 'images_table')
        find_next_img = find_img.find_all('tr')
        for td in find_next_img:
            img_tag = td.find_all('img')
            for i in img_tag:
                img_src = i.attrs['src']
                img_url_list.append(img_src) #기존 img_url_list에 추가로 append
        img_count += 20
    #image source에 접근한 후 이미지 다운로드

    
    # 요기 이상함 count = 1 위치만 바꿧는데 매우 이상해짐
    count = 0
    while count <= 10:
        for i in img_url_list:
            count += 1
            img_url = i
            resp = requests.get(i)
            file_name = series_name  + str(count) + '.jpeg'  #번호 올리면서 이미지 저장
        
            with open(file_name, 'wb') as f:
                f.write(resp.content)     
            
            

series = replace_space(input('원하는 미드를 입력하세요: '))
#get_img(series)
script_crawling(series)
save_to_txt(series)

원하는 미드를 입력하세요: breaking bad
['https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e01', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e02', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e03', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e04', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e05', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e06', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s01e07', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s02e01', 'https://www.springfieldspringfield.co.uk/view_episode_scripts.php?tv-show=breaking-bad&episode=s02e02', 'https://www.springfieldsp